In [2]:
import os
import time
import numpy as np
import tensorflow as tf
import cv2
from tqdm import tqdm

# Remove the problematic GPU memory configuration
# Instead just print GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Found {len(gpus)} GPU(s): {gpus}")
    print("Will use GPU for processing")
else:
    print("No GPU found, using CPU for processing")

def create_sr_model():
    """Create the SRGAN model architecture directly."""

    # Input layer
    inputs = tf.keras.layers.Input(shape=(None, None, 3))

    # First convolution block
    x = tf.keras.layers.Conv2D(64, kernel_size=9, padding='same')(inputs)
    x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)
    skip = x

    # Residual blocks
    for _ in range(16):
        res = x
        x = tf.keras.layers.Conv2D(64, kernel_size=3, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)
        x = tf.keras.layers.Conv2D(64, kernel_size=3, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Add()([res, x])

    # Post-residual convolution
    x = tf.keras.layers.Conv2D(64, kernel_size=3, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Add()([skip, x])

    # Upsampling blocks
    for _ in range(2):
        x = tf.keras.layers.Conv2D(256, kernel_size=3, padding='same')(x)
        x = tf.keras.layers.Lambda(lambda x: tf.nn.depth_to_space(x, 2))(x)
        x = tf.keras.layers.PReLU(shared_axes=[1, 2])(x)

    # Output layer
    x = tf.keras.layers.Conv2D(3, kernel_size=9, padding='same', activation='tanh')(x)
    x = tf.keras.layers.Lambda(lambda x: (x + 1) * 127.5)(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    return model

def enhance_frame(model, frame):
    """Enhance a single frame using the SRGAN model."""
    # Convert to RGB (OpenCV loads as BGR)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert to float and normalize to 0-1
    input_frame = rgb_frame.astype(np.float32) / 255.0

    # Add batch dimension
    input_tensor = np.expand_dims(input_frame, axis=0)

    # Get super-resolution result
    sr_tensor = model.predict(input_tensor, verbose=0)

    # Remove batch dimension and convert back to uint8
    sr_frame = np.squeeze(sr_tensor, axis=0)
    sr_frame = np.clip(sr_frame, 0, 255).astype(np.uint8)

    # Convert back to BGR for OpenCV
    sr_bgr = cv2.cvtColor(sr_frame, cv2.COLOR_RGB2BGR)

    return sr_bgr

def process_video(input_path, output_path, weights_path=None):
    """Process video with SRGAN enhancement."""
    if not os.path.exists(input_path):
        print(f"Error: Input video file not found at {input_path}")
        return False

    # Create model
    print("Creating SRGAN model...")
    model = create_sr_model()

    # Load weights if provided
    if weights_path and os.path.exists(weights_path):
        try:
            model.load_weights(weights_path)
            print(f"Successfully loaded weights from {weights_path}")
        except Exception as e:
            print(f"Warning: Failed to load weights: {e}")
            print("Proceeding with uninitialized model - results will be poor quality")
    else:
        print("No weights provided or weights file not found")
        print("Proceeding with uninitialized model - results will be poor quality")

    # Open video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {input_path}")
        return False

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Input video: {width}x{height} at {fps} FPS, {frame_count} frames")

    # New dimensions (4x upscale)
    new_width = width * 4
    new_height = height * 4
    print(f"Output video will be: {new_width}x{new_height} at {fps} FPS")

    # Create output video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))

    # Process video
    start_time = time.time()
    processed_frames = 0

    # Process frames with progress bar
    pbar = tqdm(total=frame_count, desc="Processing video")

    # Try processing just the first frame to ensure everything works
    ret, frame = cap.read()
    if ret:
        try:
            print("Testing first frame...")
            enhanced = enhance_frame(model, frame)
            out.write(enhanced)
            processed_frames += 1
            pbar.update(1)
            print(f"First frame processed successfully, dimensions: {enhanced.shape}")
        except Exception as e:
            print(f"Error processing first frame: {e}")
            cap.release()
            out.release()
            pbar.close()
            return False

    # Reset video capture
    cap.set(cv2.CAP_PROP_POS_FRAMES, 1)  # Start from second frame

    # Process the rest of the frames
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            enhanced = enhance_frame(model, frame)
            out.write(enhanced)
            processed_frames += 1
            pbar.update(1)

    except Exception as e:
        print(f"\nError during processing: {e}")
        return False

    finally:
        # Clean up
        cap.release()
        out.release()
        pbar.close()

    # Final stats
    total_time = time.time() - start_time
    print(f"\nProcessed {processed_frames} frames in {total_time:.2f} seconds")
    print(f"Average processing speed: {processed_frames / total_time:.2f} fps")
    print(f"Enhanced video saved to: {output_path}")

    return True

# For direct usage
if __name__ == "__main__":
    # Example usage
    input_video = "/content/SampleVideo_360x240_1mb.mp4"
    output_video = "/content/enhanced_output.mp4"
    weights_file = "/content/gan_generator.h5"

    process_video(input_video, output_video, weights_file)

Found 1 GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Will use GPU for processing
Creating SRGAN model...
Successfully loaded weights from /content/gan_generator.h5
Input video: 320x240 at 15.0 FPS, 205 frames
Output video will be: 1280x960 at 15.0 FPS


Processing video:   0%|          | 0/205 [00:00<?, ?it/s]

Testing first frame...


Processing video:   1%|          | 2/205 [00:10<14:50,  4.39s/it]

First frame processed successfully, dimensions: (960, 1280, 3)


Processing video: 100%|██████████| 205/205 [00:51<00:00,  4.00it/s]


Processed 205 frames in 51.24 seconds
Average processing speed: 4.00 fps
Enhanced video saved to: /content/enhanced_output.mp4
